<a href="https://colab.research.google.com/github/adamggibbs/marine-carbonate-system-ml-prediction/blob/master/data_cleaning_WCOA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# SET UP ENVIRONMENT 

# mount google drive for data storage and access
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#lib install
!pip install PYCO2SYS

Mounted at /content/drive
     |████████████████████████████████| 103 kB 25.8 MB/s 
     |████████████████████████████████| 50 kB 6.1 MB/s 
     |████████████████████████████████| 243 kB 54.6 MB/s 
  Attempting uninstall: openpyxl
    Found existing installation: openpyxl 2.5.9
    Uninstalling openpyxl-2.5.9:
      Successfully uninstalled openpyxl-2.5.9


In [ ]:
# IMPORTS

import os
import sys

import pandas as pd
import numpy as np

import PyCO2SYS as pyco2

In [ ]:
# STORE DIRECTORY WHERE DATA IS STORED

# this is an absolute path 
# and be sure to include trailing '/'
data_dir = '/content/drive/MyDrive/Adam Gibbs/data/'
wcoa_dir = data_dir + 'wcoa/'

In [ ]:
# READ DATA INTO A PANDAS DATAFRAME TO DISPLAY

print('Files in WCOA data folder:')
display(os.listdir(wcoa_dir))

dfs_wcoa= []
for file in os.listdir(wcoa_dir):
  df = pd.read_csv(wcoa_dir + file)
  dfs_wcoa.append(df.dropna().reset_index(drop=True))

print()
print("There are {0} dataframes from {0} shipboard data files from WCOA cruises".format(len(dfs_wcoa)))
print("Adjust the index below to toggle display.")
print()

display(dfs_wcoa[0])

Files in WCOA data folder:


['WCOA2016.csv']


There are 1 dataframes from 1 shipboard data files from WCOA cruises
Adjust the index below to toggle display.



,EXPOCODE,CRUISE_ID,STATION_ID,CAST_NO,NISKIN_ID,NISKIN_FLAG,SAMPLE_ID,LINE,YEAR,MONTH,DAY,TIME_UTC,LATITUDE,LONGITUDE,DEPTH_BOTTOM_METER,CTDPRS,CTDTMP,CTDSAL,SALINITY_PSS78,SALINTY_FLAG,CTDOXYGEN_UMOL_KG,OXYGEN,OXYGEN_FLAG,DIC,DIC_FLAG,TA_UMOL_KG,TA_FLAG,PH_TOT,TMP_PH_DEG_C,PH_FLAG,CARBONATE_UMOL_KG,TMP_CARBONATE_DEG_C,CARBONATE_FLAG,SILICATE_UMOL_KG,NITRATE_UMOL_KG,NITRITE_UMOL_KG,PHOSPHATE_UMOL_KG,AMMONIUM_UMOL_KG,NUTRIENTS_FLAG,CHL_A_UG_L_PC,CHL_A_UG_L_GFF,CHL_FLAG,ACCESSION
0,33RO20160505,WCOA2016,4,5,1,2,40501,1,2016,5,8,4:45:23,25.589,-113.780,2975,2500.5,1.85,34.659,34.659,2,101.5,101.9,2,2368.1,6,2434.8,2,7.4958,25,2,73.47,25,3,159.86,38.92,0.03,2.80,0.04,2,-999.00,-999.00,9,169412
1,33RO20160505,WCOA2016,4,5,2,2,40502,1,2016,5,8,4:55:58,25.589,-113.780,2975,2000.2,2.20,34.634,-999.000,9,80.2,-999.0,9,2371.3,2,2423.3,2,7.4523,25,2,67.66,25,3,152.28,40.02,0.03,2.92,0.04,2,-999.00,-999.00,9,169412
2,33RO20160505,WCOA2016,4,5,3,4,40503,1,2016,5,8,4:56:24,25.589,-113.780,2975,2000.4,2.20,34.634,-999.000,9,80.2,-999.0,9,-999.0,9,-999.0,9,-999.0000,-999,9,-999.00,25,9,-999.00,-999.00,-999.00,-999.00,-999.00,9,-999.00,-999.00,9,169412
3,33RO20160505,WCOA2016,4,5,4,4,40504,1,2016,5,8,4:56:28,25.589,-113.780,2975,1999.5,2.20,34.634,-999.000,9,80.1,-999.0,9,-999.0,9,-999.0,9,-999.0000,-999,9,-999.00,25,9,-999.00,-999.00,-999.00,-999.00,-999.00,9,-999.00,-999.00,9,169412
4,33RO20160505,WCOA2016,4,5,5,4,40505,1,2016,5,8,4:56:34,25.589,-113.780,2975,2000.8,2.20,34.634,-999.000,9,80.3,-999.0,9,-999.0,9,-999.0,9,-999.0000,-999,9,-999.00,25,9,-999.00,-999.00,-999.00,-999.00,-999.00,9,-999.00,-999.00,9,169412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2098,33RO20160505,WCOA2016,135,1,16,2,1350116,17,2016,6,6,21:56:25,50.031,-125.098,269,10.1,14.32,26.892,-999.000,9,291.6,-999.0,9,1720.1,2,1862.3,2,7.9297,25,2,135.94,25,2,7.65,1.99,0.08,0.55,0.34,2,-999.00,-999.00,9,169412
2099,33RO20160505,WCOA2016,135,1,17,2,1350117,17,2016,6,6,21:58:23,50.031,-125.098,269,5.0,14.97,26.526,-999.000,9,304.7,-999.0,9,1662.0,6,1859.4,2,8.0740,25,2,181.21,25,3,4.68,0.83,0.07,0.43,0.21,2,-999.00,-999.00,9,169412
2100,33RO20160505,WCOA2016,135,1,18,2,1350118,17,2016,6,6,21:59:36,50.031,-125.098,269,2.6,16.63,25.306,-999.000,9,303.0,303.1,2,1608.5,2,1810.2,6,8.1154,25,3,178.61,25,2,3.29,0.36,0.04,0.32,0.14,2,-999.00,-999.00,9,169412
2101,33RO20160505,WCOA2016,135,1,19,2,1350119,17,2016,6,6,21:59:39,50.031,-125.098,269,2.6,16.60,25.331,-999.000,9,303.0,-999.0,9,-999.0,9,-999.0,9,-999.0000,-999,9,-999.00,25,9,-999.00,-999.00,-999.00,-999.00,-999.00,9,1.39,3.11,2,169412


In [ ]:
# CREATE FUNCTION TO CREATE A NUMPY ARRAY OF INPUTS FROM
# SHIP DATA FILE

'''
process_wcoa_file()
  description:
    This function reads in a data file in csv format and
    creates a pandas dataframe from it. From there it loops through
    and removes all bad data points according to the quality control
    flags. It then takes the desired input parameters as sepcified on 
    line 74 and puts them into a numpy array.

  args:
    file: string that contains file name of dataset
'''
def process_wcoa_file(file):

  # read in csv
  df = pd.read_csv(file)

  # format dates
  df['DATE'] = df['YEAR'].astype('str') + '0' + df['MONTH'].astype('str')

  index = 0
  for day in df['DAY'].astype('str'):
    if len(day) == 1:
      df['DATE'][index] += '0' + day
    else:
      df['DATE'][index] += day
    index += 1

  # take subset of data we care about
  df = df[['STATION_ID', 'DATE', 'LATITUDE', 'LONGITUDE', 'CTDPRS', 'CTDTMP', 
           'CTDSAL', 'OXYGEN', 'OXYGEN_FLAG','DIC', 'DIC_FLAG', 'PH_TOT', 
           'PH_FLAG']]
  # drop any row with NaN values
  df = df.dropna(axis=0, how='any').reset_index(drop=True)

  # drop data outside our region
  index = 0
  to_drop = []
  for station in df['STATION_ID']:
    if station < 42 or station > 50:
      to_drop.append(index)
    index += 1
  df = df.drop(to_drop).reset_index(drop=True)

  # remove all bad data
  # "bad data" has a QC flag that isn't 2
  # do this for every parameter and its QC flags

  # drop bad Oxygen
  index = 0
  to_drop = []
  for flag in df['OXYGEN_FLAG']:
    if flag != 2:
      to_drop.append(index)
    index += 1
  df = df.drop(to_drop).reset_index(drop=True)

  # drop bad Total Carbon
  index = 0
  to_drop = []
  for flag in df['DIC_FLAG']:
    if flag != 2:
      to_drop.append(index)
    index += 1
  df = df.drop(to_drop).reset_index(drop=True)

  # drop bad Total pH
  index = 0
  to_drop = []
  for flag in df['PH_FLAG']:
    if flag != 2:
      to_drop.append(index)
    index += 1
  df = df.drop(to_drop).reset_index(drop=True)

  # Convert shipboard pH to in situ pH

  results = pyco2.sys(par1=df['PH_TOT'], par1_type=3,
                    par2=df['DIC'], par2_type=2,
                    temperature=25, pressure=0,
                    temperature_out=df['CTDTMP'], pressure_out=df['CTDPRS'],
                    salinity=df['CTDSAL'])

  df['PH_TOT'] = results['pH_out']

  # take subset of only parameters for inputs
  # this array contains only "good" data points
  inputs = df[['DATE', 'LATITUDE', 'LONGITUDE', 'CTDPRS', 'CTDTMP', 'CTDSAL', 
               'OXYGEN']]
  outputs = df['PH_TOT']

  # convert dataframe in numpy array
  inputs = inputs.to_numpy(dtype='str')
  outputs = outputs.to_numpy(dtype='str')

  # return the array
  return inputs, outputs


'''
process_wcoa_dir()
  description:
    This function takes a dir with shipboard data and processes
    each file in that dir and creates input and output files. It
    also has the option to save those input and output arrays to
    either a .txt or a .csv file.

  args:
    file: string that contains file name of dataset
    save_txt (default=False): boolean of whether to save a .txt
    save_csv (default=False): boolean of whether to save a .csv
'''
def process_wcoa_dir(dir, save_txt=False, save_csv=False):
  # list all data files in directory
  print('Parsing the following files:')
  print(os.listdir(dir))

  # create an empty numpy array that will hold inputs
  inputs = np.empty((0,7))
  outputs = np.empty(0)

  # loop through all data files and add them to input array
  for file in os.listdir(dir):
    input_array, output_array = process_wcoa_file(dir + file)
    inputs = np.concatenate((inputs, input_array), axis=0)
    outputs = np.concatenate((outputs, output_array), axis=0)

  # if desired save arrays as .txt files
  if save_txt or save_csv: 
    if save_txt:
      input_header = 'DATE LATITUDE LONGITUDE CTDPRS CTDTMP CTDSAL OXYGEN'
      np.savetxt(data_dir + 'wcoa_tpso_input.txt', inputs, 
                 fmt='%s', header=input_header)
      output_header = 'TOT_PH'
      np.savetxt(data_dir + 'wcoa_ph_output.txt', outputs, 
                 fmt='%s', header=output_header)
    else:
      input_header = 'DATE, LATITUDE, LONGITUDE, CTDPRS, CTDTMP, CTDSAL, OXYGEN'
      np.savetxt(data_dir + 'wcoa_tpso_input.csv', inputs, 
                 fmt='%s', delimiter=",", header=input_header)
      output_header = 'TOT_PH'
      np.savetxt(data_dir + 'wcoa_ph_output.csv', outputs, 
                 fmt='%s', delimiter=",", header=output_header)

  return inputs, outputs


In [ ]:
# PARSE WCOA DATA AND CREATE INPUT ARRAY

wcoa_inputs, wcoa_outputs = process_wcoa_dir(wcoa_dir, save_csv=True)

# print input array and its dimensions
print('Input array:')
display(wcoa_inputs)

print('Dimensions of input array:')
display(wcoa_inputs.shape)

# print output array and its dimensions
print('Output array:')
display(wcoa_outputs)

print('Dimensions of output array:')
display(wcoa_outputs.shape)

Parsing the following files:
['WCOA2016.csv']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Input array:


array([['20160518', '36.81', '-121.818', '15.0', '13.34', '33.143',
        '272.8'],
       ['20160518', '36.81', '-121.818', '5.1', '13.96',
        '33.138000000000005', '273.9'],
       ['20160518', '36.81', '-121.818', '3.4', '14.0', '33.139',
        '274.0'],
       ['20160518', '36.795', '-121.848', '251.2', '7.89', '34.05',
        '67.8'],
       ['20160518', '36.795', '-121.848', '151.0', '8.62', '33.954',
        '92.6'],
       ['20160518', '36.795', '-121.848', '100.5', '8.87', '33.908',
        '104.4'],
       ['20160518', '36.795', '-121.848', '40.8', '10.09', '33.617',
        '165.0'],
       ['20160518', '36.795', '-121.848', '30.4', '10.17', '33.548',
        '177.7'],
       ['20160518', '36.795', '-121.848', '20.0', '10.53', '33.399',
        '190.5'],
       ['20160518', '36.795', '-121.848', '10.5', '12.65', '33.321',
        '238.8'],
       ['20160518', '36.784', '-121.87200000000001', '100.2', '8.67',
        '33.959', '91.2'],
       ['20160518', '36.784', 

Dimensions of input array:


(48, 7)

Output array:


array(['8.034862400919026', '8.040221578006799', '8.042729240792925',
       '7.614367250629397', '7.649846263031173', '7.666234175745569',
       '7.759884604571789', '7.791583909488228', '7.829769056186604',
       '7.9571426289466585', '7.654406486127567', '7.731111957408882',
       '7.792791213405503', '7.844613316074569', '7.934089886224436',
       '7.728975840249242', '7.813043343673816', '7.654713691902718',
       '7.734461970398188', '7.783192154238925', '7.874654237803034',
       '8.020945551070671', '7.547830986271524', '7.587966406959314',
       '7.661293458326776', '7.704925720936962', '8.028133557088248',
       '7.548931548136637', '7.559850661425167', '7.627537898323465',
       '7.689201872403516', '7.7141705029480026', '7.825143189743733',
       '7.889543331973465', '7.625734472182768', '7.552612173292936',
       '7.640436648343181', '7.717278360710171', '7.82518714006605',
       '7.914619909617428', '7.994079254751035', '7.631890490116242',
       '7.538796826

Dimensions of output array:


(48,)